In [21]:
class Unifier:
    def is_var(self, sym):
        """Helper function to check if a symbol is a variable (a string)."""
        return isinstance(sym, str) and sym.islower()

    def occurs_check(self, var, term):
        """Checks if a variable occurs in a term (used to avoid circular dependencies)."""
        if isinstance(term, str):
            return term == var
        elif isinstance(term, list):
            return any(self.occurs_check(var, t) for t in term)
        elif isinstance(term, tuple):
            return any(self.occurs_check(var, t) for t in term)
        return False

    def unify_var(self, var, val, subst):
        """Unifies a variable with a value, handling substitution."""
        if var in subst:
            return self.unify(subst[var], val, subst)  # Recurse with the substituted value
        elif isinstance(val, str) and val in subst:
            return self.unify(var, subst[val], subst)  # Recurse with the substituted value of val
        elif self.occurs_check(var, val):  # Check for circular dependency
            return False
        else:
            subst[var] = val  # Apply the substitution
            return subst

    def unify(self, sym1, sym2, subst):
        """Unifies two symbols or terms."""
        if subst is False:
            return False

        # Case 1: Both symbols are the same (unification is trivially successful)
        elif isinstance(sym1, str) and isinstance(sym2, str) and sym1 == sym2:
            return subst

        # Case 2: One of the symbols is a variable
        elif isinstance(sym1, str) and self.is_var(sym1):
            return self.unify_var(sym1, sym2, subst)

        elif isinstance(sym2, str) and self.is_var(sym2):
            return self.unify_var(sym2, sym1, subst)

        # Case 3: Both symbols are compound terms (tuples or lists)
        elif isinstance(sym1, tuple) and isinstance(sym2, tuple):  # Predicate case
            if len(sym1) == 0 and len(sym2) == 0:
                return subst  # Both are empty tuples
            if sym1[0] != sym2[0]:  # Predicate symbols (first elements) must match
                return False
            return self.unify(sym1[1:], sym2[1:], self.unify(sym1[0], sym2[0], subst))

        elif isinstance(sym1, list) and isinstance(sym2, list):  # List case
            if len(sym1) == 0 and len(sym2) == 0:
                return subst  # Both are empty lists
            return self.unify(sym1[1:], sym2[1:], self.unify(sym1[0], sym2[0], subst))

        # If no cases match, return failure
        else:
            return False

# Example usage:
unifier = Unifier()

ψ1 = ['f', 'x', 'x']
ψ2 = ['f', 'Z', 'f(Z)']
substitution = unifier.unify(ψ1, ψ2, {})

if substitution is False:
    print("FAILURE")
else:
    print("Substitution:", substitution)


FAILURE
